In [24]:
# import the libraries
import requests
import csv
from bs4 import BeautifulSoup

In [27]:
# convert model names to brands
import csv

# open the models file and use that to replace model names with brand names. 
# format of file is brand, model
file = open('models.csv', 'r')
csv_file = csv.reader(file)

# store the brand names
brands = []
for i in csv_file:
    brands.append(i)
#     parsedData = parsedData.replace(i[1], i[0])

print('Finished storing this csv file into a list')
# close the file
file.close()

Finished storing this csv file into a list


In [26]:
brands

[['acura', 'integra'],
 ['acura', 'Legend\xa0'],
 ['acura', 'vigor'],
 ['acura', 'rlx'],
 ['acura', 'ILX'],
 ['acura', 'MDX'],
 ['acura', 'RDX'],
 ['acura', 'TSX'],
 ['acura', 'RSX'],
 ['audi', 'quattro'],
 ['audi', 'a4'],
 ['audi', 'a6'],
 ['audi', 'a5'],
 ['audi', 'a7'],
 ['audi', 'q3'],
 ['audi', 'q5'],
 ['audi', 'q7'],
 ['audi', 'a8'],
 ['audi', 'a3'],
 ['bmw', '3-series'],
 ['bmw', '5-series'],
 ['bmw', '6-series'],
 ['bmw', '4-series'],
 ['bmw', 'xdrive'],
 ['bmw', 'alpina'],
 ['bmw', '330ci'],
 ['bmw', '328i'],
 ['bmw', '330c'],
 ['bmw', '335i'],
 ['bmw', '335d'],
 ['bmw', '528i'],
 ['bmw', '535i'],
 ['bmw', '128i'],
 ['bmw', '640i'],
 ['bmw', '3er'],
 ['bmw', '5er'],
 ['bmw', 'M5'],
 ['bmw', 'M3'],
 ['bmw', 'M6'],
 ['bmw', 'bmws'],
 ['bmw', 'bmwesque'],
 ['buick', 'park avenue'],
 ['buick', 'la crosse'],
 ['buick', 'lacrosse'],
 ['buick', 'century'],
 ['buick', 'enclave'],
 ['buick', 'encore'],
 ['buick', 'regal'],
 ['buick', 'buicks'],
 ['cadillac', 'escalade'],
 ['cadillac', 

## Scraping all pages

In [2]:
# we'll be using the cars and conversations forum
url = 'https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations'

In [3]:
# number of forum pages
num_pages = 2572

In [28]:
# store the comments in a string
all_comments = ""

# store output: date, userid, message
output_csv = []

# go through each page
for page_number in range(1, 3):
    # first page string of url isn't anything special
    try:
        if(page_number == 1):
            new_url = url
        else:
            # end url format is /p220 etc.
            new_url = url + "/p" + str(page_number)
        print(new_url)
        # collect the url with requests library
        page = requests.get(new_url)
    
        # get the html of the page in string form
        page_html = page.text
    
        # create the BeautifulSoup object that takes in the html in str form and a html/xml parser of choice either html.parser or lxml
        soup = BeautifulSoup(page_html, 'html.parser')
    
        # inspect the elements from the webpage to know where stuff is
        # in this case the comments are under a <p> tag which are all under a <div class = "Message userContent"</div/ tag 
        # which are all under a <div class = "MessageList DataList Comments"> tag
    
        # Pull all text from the 'MessageList DataList Comments' divs
        messagelist = soup.find(class_ = "MessageList DataList Comments")
        
        # within the 'MessageList DataList Comments' divs pull all text from 'Message userContent' divs
        #usermessages = messagelist.find_all(class_ =  "Message userContent")
        usermessages = messagelist.find_all(class_ = "Comment")
        
        # now extract just the <p> tags from all comments! 
        for i in range(len(usermessages)):
            # let's grab the date and username of the post which is under class = "Item-Header CommentHeader"
            # in a <time> tag and class = "Username"
            commentheader = usermessages[i].find(class_ = 'Item-Header CommentHeader')
            
            # grab the date. Format: June 18, 2014  3:53PM is stored in title = ' ' of <time  >
            date_of_post = commentheader.find("time", title = True)
            
            # variable to store date
            date_of_post = date_of_post['title']
            
            # grab the username
            username = commentheader.find(class_ = "Username")
            # variable to store username
            username = username.string
            
            # now grab text under Message userContent
            messageheader = usermessages[i].find(class_ = 'Message userContent')
            
            # a user comment might have multiple <p> tags
            # [s.extract() for s in usermessages[i](class_ = 'UserQuote')]
            [s.extract() for s in messageheader(class_ = 'UserQuote')]
            user_comments = messageheader.find_all('p')
            
            # variable to store user message
            message = ""
            for j in range(len(user_comments)):
                # remove <a>, <img>, <br> tags embedded in <p>
                [s.extract() for s in user_comments[j]('a')]
                [s.extract() for s in user_comments[j]('img')]
                [s.extract() for s in user_comments[j]('br')]

                
                # Let's do some string manipulation 
                # remove the newline characters, '<p>', '</p','said:'  
                # add the users comments to the mega string

                message += user_comments[j].prettify()

            # let's do some string manipulation
            # remove the newline characters, '<p>', '</p','said:'
            message = message.replace('\n', '')
            message = message.replace(r"<p>", "")
            message = message.replace(r"</p>", "")
            message = message.replace(r"</p>", "")
            message = message.replace(r"said:", "")
            message = message.replace(r":", "")
            message = message.lower()
            
            # deal with brand name
            for k in brands:
                message = message.replace(k[1], k[0])

            # add to all_comments
            all_comments += message
            
            # create a tuple with Date, Userid, Message and store into output_csv
            row = date_of_post, username, message
            output_csv.append(row)
            
            # print this if you want to see some output
            # print('username: ' + str(username))
            # print('date: ' + str(date_of_post))
            # print('message: ' + message)
            
        # print some checks
        print('Page ' + str(page_number) + ' complete')
    except:
        print('page ' + str(page_number) + ' failed')
        continue

https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations
Page 1 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2
Page 2 complete


## Throw output_csv list into a csv file and place all_text into a text file

In [29]:
# output into csv file



In [11]:
# throw this into a text file just in case
new_file = open('alltext.txt', 'w', encoding = 'utf-8')
new_file.write(parsedData)
new_file.close()

## Word Frequency

In [13]:
# grab word frequency using nltk library
import nltk
# stop words
from nltk.corpus import stopwords
nltk.download()
nltk.download('stopwords')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# tokenize the text 
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(parsedData)

In [15]:
# let's do a word count
from collections import Counter

# counter object
words = Counter()

# update counter with new words
words.update(tokens)

In [21]:
# check
words.most_common()[:100]

[('car', 2209),
 ('one', 987),
 ('like', 944),
 ('would', 907),
 ('get', 730),
 ('new', 662),
 ('think', 568),
 ('good', 511),
 ('back', 448),
 ('time', 446),
 ('know', 429),
 ('years', 416),
 ('dealer', 409),
 ('benz', 405),
 ('could', 401),
 ('much', 395),
 ('people', 389),
 ('ford', 378),
 ('buy', 369),
 ('well', 367),
 ('even', 358),
 ('see', 346),
 ('got', 330),
 ('year', 325),
 ('want', 320),
 ('go', 318),
 ('said', 316),
 ('really', 315),
 ('way', 301),
 ('going', 293),
 ('price', 287),
 ('make', 283),
 ('still', 269),
 ('better', 269),
 ('right', 266),
 ('never', 261),
 ('many', 260),
 ('also', 259),
 ('used', 249),
 ('bmw', 246),
 ('money', 241),
 ('take', 236),
 ('last', 231),
 ('day', 229),
 ('lot', 227),
 ('nice', 227),
 ('old', 225),
 ('first', 225),
 ('gm', 225),
 ('sure', 224),
 ('two', 219),
 ('drive', 218),
 ('problem', 214),
 ('may', 210),
 ('every', 210),
 ('something', 206),
 ('us', 204),
 ('mercedes', 202),
 ('miles', 201),
 ('probably', 197),
 ('use', 195),
 ('loo

So these are the most common words, but they are also the same in any english text. Therefore they are not very insightful. Hence, let's remove single characters, numbers, and common words.

In [17]:
# store the stop words
stopwords = stopwords.words('english')[:]

In [18]:
# lets do this again: remove stop words, single letter variable names
words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1)

In [20]:
# check
words.most_common()[:1000]

[('car', 2209),
 ('one', 987),
 ('like', 944),
 ('would', 907),
 ('get', 730),
 ('new', 662),
 ('think', 568),
 ('good', 511),
 ('back', 448),
 ('time', 446),
 ('know', 429),
 ('years', 416),
 ('dealer', 409),
 ('benz', 405),
 ('could', 401),
 ('much', 395),
 ('people', 389),
 ('ford', 378),
 ('buy', 369),
 ('well', 367),
 ('even', 358),
 ('see', 346),
 ('got', 330),
 ('year', 325),
 ('want', 320),
 ('go', 318),
 ('said', 316),
 ('really', 315),
 ('way', 301),
 ('going', 293),
 ('price', 287),
 ('make', 283),
 ('still', 269),
 ('better', 269),
 ('right', 266),
 ('never', 261),
 ('many', 260),
 ('also', 259),
 ('used', 249),
 ('bmw', 246),
 ('money', 241),
 ('take', 236),
 ('last', 231),
 ('day', 229),
 ('lot', 227),
 ('nice', 227),
 ('old', 225),
 ('first', 225),
 ('gm', 225),
 ('sure', 224),
 ('two', 219),
 ('drive', 218),
 ('problem', 214),
 ('may', 210),
 ('every', 210),
 ('something', 206),
 ('us', 204),
 ('mercedes', 202),
 ('miles', 201),
 ('probably', 197),
 ('use', 195),
 ('loo

In [22]:
# now let's do this again but with only brand names
words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1 and x in brands)

In [24]:
brands

['acura',
 'audi',
 'bmw',
 'buick',
 'cadillac',
 'car',
 'chevrolet',
 'chrysler',
 'dodge',
 'ford',
 'honda',
 'hyndai kia',
 'hyundai',
 'hyundai,',
 'hyundai.',
 'infiniti',
 'kia',
 'kia,',
 'kia.',
 'lincoln',
 'mazda',
 'mercedes',
 'mercedes benz',
 'mercedes-benz',
 'mercury',
 'mitsubishi',
 'nissan',
 'nissan.',
 'pontiac',
 'problem',
 'saturn',
 'seat',
 'sedan',
 'subaru',
 'suzuki',
 'toyata',
 'toyota',
 'volkswagen',
 'volkwagen',
 'volvo']

In [23]:
# check
words.most_common()[:]

[('car', 2209),
 ('ford', 378),
 ('bmw', 246),
 ('problem', 214),
 ('mercedes', 202),
 ('chevrolet', 172),
 ('buick', 143),
 ('audi', 136),
 ('volkswagen', 135),
 ('seat', 126),
 ('cadillac', 113),
 ('honda', 111),
 ('toyota', 109),
 ('dodge', 82),
 ('chrysler', 81),
 ('subaru', 77),
 ('acura', 67),
 ('nissan', 62),
 ('sedan', 59),
 ('lincoln', 58),
 ('hyundai', 52),
 ('mazda', 47),
 ('volvo', 39),
 ('kia', 20),
 ('infiniti', 9),
 ('pontiac', 9),
 ('saturn', 8),
 ('mitsubishi', 5),
 ('mercury', 3)]